## Mobile APP using Yolo Model

* Coders:
    - Lenin Falconí
    - Mario Moreno
    - Jonathan Zea

## Goals

- Obtener un modelo en /TFLITE/ para consumir modelos de Yolo en sistemas móbiles como Android
- Explorar las predicciones con Yolo

In [ ]:
import torch
from torchvision.models import yolov8
model = yolov8(pretrained=True)
model.export('model.tflite')